In [ ]:
!pip install web3

In [ ]:
from web3 import Web3

w3 = Web3(Web3.HTTPProvider(
    'https://goerli.infura.io/v3/<Project_ID>'))
w3.isConnected()

In [ ]:
w3.eth.get_block_number()

In [ ]:
block = w3.eth.get_block('latest')
block

In [ ]:
block.number

In [ ]:
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os 

load_dotenv()

# Account 1
account1_address = '0xAF8b6CA21023A595F0C4919b8B4a9d1F0c1773e7'
account1_private_key = os.environ.get('account1_private_key')

# Account 2
account2_address = '0x63eE1AEb74c52f09EaB6a2825bB1918B5e045050'

In [ ]:
w3.eth.get_balance(account1_address)

In [ ]:
nonce = w3.eth.get_transaction_count(account1_address)
tx = {
    'nonce': nonce,                      # transaction count
    'to': account2_address,              # who to send the ETH to
    'value': w3.toWei(1000, 'wei'),      # the amount to transfer
    'gasPrice': w3.eth.gas_price,        # get the price of gas
}

In [ ]:
gas = w3.eth.estimate_gas(tx)
tx['gas'] = gas
print(tx)

In [ ]:
signed_tx = w3.eth.account.sign_transaction(tx,account1_private_key)

In [ ]:
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
print(w3.toHex(tx_hash))

In [ ]:
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [ ]:
print(w3.eth.get_balance(account1_address))
print(w3.eth.get_balance(account2_address))

In [ ]:
nonce = w3.eth.get_transaction_count(account1_address)
tx = {
    'nonce': nonce,
    'to': account2_address,
    'value': w3.toWei(1000, 'wei'),
    'gasPrice': w3.eth.gas_price,
}
gas = w3.eth.estimate_gas(tx)
tx['gas'] = gas
signed_tx = w3.eth.account.sign_transaction(tx,account1_private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

print(w3.toHex(tx_hash))

receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [ ]:
address = '0xC1B4338a54bE22067260bbA1e0B6F3d5c1E2E330'
abi = '[  {   "inputs": [],   "name": "cashOut",   "outputs": [],   "stateMutability": "nonpayable",   "type": "function"  },  {   "inputs": [    {     "internalType": "string",     "name": "document",     "type": "string"    }   ],   "name": "checkEduCredentials",   "outputs": [],   "stateMutability": "payable",   "type": "function"  },  {   "inputs": [],   "name": "kill",   "outputs": [],   "stateMutability": "nonpayable",   "type": "function"  },  {   "anonymous": false,   "inputs": [    {     "indexed": false,     "internalType": "address",     "name": "from",     "type": "address"    },    {     "indexed": false,     "internalType": "string",     "name": "document",     "type": "string"    },    {     "indexed": false,     "internalType": "uint256",     "name": "blockNumber",     "type": "uint256"    }   ],   "name": "Result",   "type": "event"  },  {   "inputs": [    {     "internalType": "string",     "name": "document",     "type": "string"    }   ],   "name": "storeEduCredentials",   "outputs": [],   "stateMutability": "nonpayable",   "type": "function"  } ]'
eduCredentialsStore = w3.eth.contract(address = address, abi = abi)

In [ ]:
import base64

def base64encode(message):
    message_bytes = message.encode('ascii')
    base64_bytes = base64.b64encode(message_bytes)
    return base64_bytes.decode('ascii')

In [ ]:
exam_result = '''
{
  "id": "1234567",
  "result": {
    "math": "A",
    "science": "B",
    "english": "A"
  }
}
'''

exam_result = base64encode(exam_result)
nonce = w3.eth.get_transaction_count(account1_address)

# estimate the gas fee
estimated_gas = \
    eduCredentialsStore.functions.storeEduCredentials(
        exam_result).estimateGas(
            {'from':account1_address})

# build the transaction
transaction = \
  eduCredentialsStore.functions.storeEduCredentials(
    exam_result).buildTransaction(
    {
        'gas': estimated_gas,
        'gasPrice': w3.eth.gas_price,
        'from': account1_address,
        'nonce': nonce
    })

# sign the transaction
signed_txn = w3.eth.account.sign_transaction(
    transaction, private_key = account1_private_key)

# send the transaction
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(w3.toHex(tx_hash))

# wait for the transaction to confirm
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
receipt

In [ ]:
exam_result = '''
{
  "id": "1234567000",
  "result": {
    "math": "A",
    "science": "B",
    "english": "A"
  }
}
'''
exam_result = base64encode(exam_result)

nonce = w3.eth.getTransactionCount(account1_address)

# -------------------------------------------------
# do this if there is no transaction involved with the function
# eduCredentialsStore.functions.checkEduCredentials(exam_result).call()
# -------------------------------------------------

# estimate the gas fee
estimated_gas = eduCredentialsStore.functions.checkEduCredentials(
                    exam_result).estimateGas(
                        { 'value' : 1000 } 
                    )  # 1000 is the wei to send                                     
    
# build the transaction
transaction = eduCredentialsStore.functions.checkEduCredentials(
    exam_result).buildTransaction(
    {
        'gas'      : estimated_gas,
        'gasPrice' : w3.eth.gas_price,
        'from'     : account1_address,
        'nonce'    : nonce,
        'value'    : w3.toWei(1000, 'wei'),   # amount to send to the 
    })                                        # function

# sign the transaction
signed_txn = w3.eth.account.sign_transaction(transaction, 
             private_key = account1_private_key)

# send the transaction
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(w3.toHex(tx_hash))

import time

# create an instance of the event
result_event = eduCredentialsStore.events.Result()

def handle_event(event):
    receipt = \
       w3.eth.wait_for_transaction_receipt(event['transactionHash'])
    result = result_event.processReceipt(receipt)    
    
    # print the content of the Result event
    print(result)
    if result[0]['args']['blockNumber'] != 0:
        print('Result is verified.')
    else:
        print('Result not found on blockchain.')
    return True

def log_loop(event_filter, poll_interval):
    while True:
        for event in event_filter.get_new_entries():
            result = handle_event(event)
            if result == True: 
                return
            time.sleep(poll_interval)
            
block_filter = w3.eth.filter(
    {
        'fromBlock' : 'latest', 
        'address'   : address       # address of contract
    })

log_loop(block_filter, 2)